# 1. Insert Boolean logic text
### (1) Spacing is required between parentheses, names, and logic operators.
    
    ex) "A = ( B and ( not C or D ) ) and E"

### (2) Set the state ("True" or "False") of the input nodes.

    ex) "A = False"

In [1]:
# Boolean network example in figure 4
booleanlogic = '''
A = D
B = not A
C = A and not B
D = C
E = C or F or G
F = not E
G = G or F
H = D or I
I = F or H
'''

# 2. Preprocessing Boolean logic text
### change node names into number nodes
(Please check booleanlogicNum before performing the following steps to avoid possible errors.)

In [2]:
from dcgs import booleanlogic_preprocessing

booleanlogicNum, node2num, num2node = booleanlogic_preprocessing.modeltext2nummodeltext(booleanlogic)
dgraph, nodeList, inputNodeState = booleanlogic_preprocessing.get_interaction_network(booleanlogicNum)
print(booleanlogicNum)


n1 = n4
n2 = not n1
n3 = n1 and not n2
n4 = n3
n5 = n3 or n6 or n7
n6 = not n5
n7 = n7 or n6
n8 = n4 or n9
n9 = n6 or n8



# 3. Get minimal feedback vertex sets (FVSs)
### get FVSs of each strongly connected component and combine them.

In [3]:
from dcgs import sccTofvs
dic_hierarchy, dic_fvs, minimal_fvs = sccTofvs.scc2fvs_bruteforce(dgraph)
print('FVS of each SCC: ',dic_fvs)
print('FVS example: ',minimal_fvs)

FVS of each SCC:  {'n8_n9': [['n8'], ['n9']], 'n7_n5_n6': [['n7', 'n5'], ['n7', 'n6']], 'n1_n3_n2_n4': [['n1'], ['n3'], ['n4']]}
FVS example:  ['n1', 'n5', 'n7', 'n8']


# 4. Get point attractors
### get point attractors from canalized state set of FVS

In [4]:
from dcgs import attractorlandscapeSeeker
CSS, state_origin = attractorlandscapeSeeker.get_canalized_states(booleanlogicNum, dgraph, nodeList, inputNodeState, [minimal_fvs])
paStr,paDic = attractorlandscapeSeeker.get_pointattractorFromCSS(booleanlogicNum, CSS)
print('Point attractors: ',paStr)

Point attractors:  ['010010100', '010010111', '101110011', '101110111']


# 5. Get control sets for global stabilization using DCGS
### get control sets (canalizing sets) for global stabilization of a target attractor

In [5]:
from dcgs import Main
targetattractor = paStr[3]
dic_CS,dic_cCS, dic_fvs = Main.algorithm(booleanlogicNum, targetattractor)
print('control sets (canalizing sets) for each FVS of each SCC: ',dic_CS)

control sets (canalizing sets) for each FVS of each SCC:  {'n1_n3_n2_n4': [['n1'], ['n3'], ['n4'], ['n1'], ['n3'], ['n4'], ['n1'], ['n3'], ['n4'], ['n1'], ['n3'], ['n4']], 'n7_n5_n6': [['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7'], ['n7']], 'n8_n9': [[]]}


### aggregate canalizing sets of each SCC

In [6]:
import copy
controlSet = []
for scc in dic_CS:
    csList = dic_CS[scc]
    if csList == [[]]:
        continue
    if controlSet == []:
        controlSet = csList.copy()
        continue
    newcontrolSetlist = []
    controlSet_copy = controlSet.copy()
    for cs in csList:
        for cs_origin in controlSet_copy:
            newcontrolSetlist.append(cs_origin+cs)
    controlSet = newcontrolSetlist.copy()
controlset_filtered = []
for x in controlSet:
    if x not in controlset_filtered:
        controlset_filtered.append(x)
print(controlset_filtered)

[['n1', 'n7'], ['n3', 'n7'], ['n4', 'n7']]


# 6. Compare the canalizing sets with the CKs from the brute force CK algorithm

In [7]:
from dcgs import bruteforceCK
CK = bruteforceCK.algorithm(booleanlogicNum, targetattractor)
print(CK)

[('n1', 'n7'), ('n7', 'n4'), ('n7', 'n3')]
